# SnowExSQL Database 


[SnowEx](https://snow.nasa.gov/campaigns/snowex) has introduced a unique opportunity to study SWE in a way that's unprecedented, but with more data comes new challenges. 

![examples](./images/snowex_database/data_examples.png)


<!-- 
<img src="https://snowexsql.readthedocs.io/en/latest/_images/gallery_overview_example_12_0.png" alt="Grand Mesa Overview" width="1000px"> -->

**The SnowEx database is a resource that shortcuts the time it takes to ask cross dataset questions**

      
- Standardizing diverse data
- Cross referencing data
- Provenance!
- Added GIS functionality
- Connect w/ ArcGIS or QGIS!
      
      

## What's in it?

* Snow pits - Density, hardness profiles, grain types + sizes
* Manual snow depths - TONS of depths (Can you say spirals?)
* Snow Micropenetrometer (SMP) profiles - (Subsampled to every 100th)
* Snow depth + SWE rasters from ASO Inc.
* GPR
* Pit site notes
* Camera Derived snow depths
* Snow off DEM from USGS 3DEP 
* And almost all the associated metadata

## Technically, what is it?

* PostgreSQL database
* PostGIS extension
* Supports vector and raster data
* And a host of GIS operations
* AND NOW WITH API!


### So what's the catch?
New tech can create barriers...

```{figure} ./images/snowex_database/pits_not_bits.jpg
:scale: 20 %
:alt: pits not bits
```

### TL;DR Do less wrangling, do more crunching. 

## How do I get at this magical box of data ?
* [SQL](https://www.postgresql.org/docs/13/tutorial-sql.html) 
* [snowexsql](https://github.com/SnowEx/snowexsql/) <span style="font-size:20pt;"> **&#8592; 😎**</span>


### Welcome to API Land

In [ ]:
from snowexsql.api import PointMeasurements

df = PointMeasurements.from_filter(type="depth", instrument='pit ruler', limit=100)
df.plot(column='value', cmap='jet', vmin=10, vmax=150)
df